In [149]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.kaleido.scope.mathjax = None

# Template

In [150]:
matlab_colors = ['#0072BD', '#D95319', '#EDB120', '#7E2F8E', '#77AC30', '#4DBEEE', '#A2142F']

In [151]:
# Constants factors
margin_inches = 1/16
ppi = 300
default_ppi = 96
scale = ppi / default_ppi

width_inches = 3.5
height_inches = 2/3 * 3.5
width = (width_inches - margin_inches) * ppi
height = (height_inches - margin_inches) * ppi

font_size = scale * 8.25

In [152]:
margin = margin_inches * ppi

paper_style = dict(
    width=width,
    height=height,
    # title=dict(
    #     text="Title",
    #     x=0.5
    # ),
    xaxis=dict(
        ticks="inside",
        title="x",
    ),
    yaxis=dict(
        ticks="inside",
        title="y",
    ),
    template="simple_white",
    font=dict(
        family="Linux Biolinum O Regular",
        size=font_size
    ),
    legend=dict(
        font=dict(
            family="Linux Biolinum O Regular",
            size=font_size
        ),
        orientation="h",
        yanchor="top",
        y=1.05,
        xanchor="center",
        x=0.5,
        bgcolor='rgba(0,0,0,0)'
    ),
    margin=dict(l=margin, r=margin, t=6*margin, b=margin)
    
)

# Parse Log

In [153]:
log_file = "/home/zjf214/scratch/remeshing-project/results/ball-wall/3D/2023_01_21_07_38_53_655/log.txt"
with open(log_file) as f:
    log = f.readlines()

In [154]:
timestep_end = []
for i, line in enumerate(log):
    if "/200  t=" in line:
        timestep_end.append(i)

In [155]:
splits = []
collapses = []
prev_i = 0
for i in timestep_end:
    for line in log[prev_i:i]:
        if "[split]" in line:
            splits.append(line)
        elif "[collapse]" in line:
            collapses.append(line)
    prev_i = i
len(splits), len(collapses)

(200, 200)

In [156]:
n_vertices_per_timestep = []
found_prev_dof = False
for line in log:
    if "# DOF" in line and "Avg." not in line:
        # [2023-01-21 07:38:55.513] [polyfem] [critical] # DOF: 5292
        try:
            n_vertices_per_timestep.append(int(line.split(":")[-1].strip())//3)
        except:
            print(line.strip())
        found_prev_dof = True
    elif "/200  t=" in line:
        if not found_prev_dof:
            n_vertices_per_timestep.append(n_vertices_per_timestep[-1])
        found_prev_dof = False

In [157]:
aggregate_split_success = np.array([int(split.split()[6]) for split in splits])
aggregate_split_fail = np.array([int(split.split()[8]) for split in splits])
aggregate_collapse_success = np.array([int(collapse.split()[6]) for collapse in collapses])
aggregate_collapse_fail = np.array([int(collapse.split()[8]) for collapse in collapses])

In [158]:
split_per_timestep = aggregate_split_success[1:] - aggregate_split_success[:-1]
collapse_per_timestep = aggregate_collapse_success[1:] - aggregate_collapse_success[:-1]

In [159]:
#

# Parse Energy

In [160]:
energy_file = "/home/zjf214/scratch/remeshing-project/results/ball-wall/3D/2023_01_21_07_38_53_655/energy.csv"
df = pd.read_csv(energy_file)

In [161]:
start_energy = np.array(df["total_energy"][1::3])
end_energy = np.array(df["total_energy"][::3])
delta_energy = start_energy - end_energy

In [162]:
r = 0.5 * 0.04
vol = 4/3 * np.pi * r**3
rho = 1150
m = rho * vol
v = 67
initial_energy = 1/2 * m * v**2

# Plot

In [168]:
t = np.arange(200) * 2e-5

# Create figure with secondary y-axis
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x=t, y=split_per_timestep, name="# Splits", marker_color=matlab_colors[0]),
    secondary_y=True,
)
fig.add_trace(
    go.Bar(x=t, y=-collapse_per_timestep, name="# Collapses", marker_color=matlab_colors[1]),
    secondary_y=True,
)

# delta_v = np.array(n_vertices_per_timestep[1:]) - np.array(n_vertices_per_timestep[:-1])
fig.add_trace(
    # go.Scatter(x=t, y=n_vertices_per_timestep, name="# Vertices"),
    go.Scatter(x=t, y=delta_energy, name="E_t(x', T_t)", line_color=matlab_colors[2]),
    # go.Scatter(x=t, y=delta_v, name="Δ # Vertices"),
    secondary_y=False,
)

# Add figure title
fig.update_layout(**paper_style)

# Set x-axis title
fig.update_xaxes(title_text="t (s)")

def refletive_range(x):
    max_x = 1.1 * max(abs(x))
    return [-max_x, max_x]

# Set y-axes titles
max_op = np.array([max(split_per_timestep), max(collapse_per_timestep)])
fig.update_yaxes(title_text="", secondary_y=True, visible=False, range=refletive_range(max_op))
fig.update_yaxes(title_text="ΔE", secondary_y=False, range=refletive_range(delta_energy))
# fig.update_yaxes(title_text="# Vertices", secondary_y=False, range=[-max(delta_v), max(delta_v)])

fig.show()

In [164]:
# Write figure with dimensions in inches
pio.write_image(
    fig, 'am_energy.pdf',
    width=width,
    height=height,
    scale=1/scale)

In [165]:
# from IPython.display import IFrame, display
# filepath = "am_energy.pdf"
# IFrame(filepath, width=975, height=350)